In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [7]:
data = pd.read_csv('drugsComTrain_raw.tsv',sep="\t")
# data = data.sample(frac = 1)
#this data contains 160000 rows so to save time for preprocessing we are considering first 1000 rows only
data = data.iloc[:1000,:]


In [8]:
data.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [9]:
data.drop(["Unnamed: 0","date"],axis=1,inplace=True)

In [10]:
data.head()

,drugName,condition,review,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,37


In [11]:
data.shape

(1000, 5)

In [12]:
# first review less than 2 rating
print(data[data['rating']<2.0].iloc[0:1,:]["review"])

6    "He pulled out, but he cummed a bit in me. I t...
Name: review, dtype: object


In [13]:
print(data['review'][6])

"He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I&#039;m pregnant."


In [14]:
import pickle
import keras
loadedtokenizer = pickle.load(open('imdb_tokenizer.pkl','rb'))
loadedmodel= keras.models.load_model('imdb_review_feedback_emotion.h5')
# global loadedmodel,loadedtokenizer

In [15]:
def positivity_score(text):
    data4 = [text]
    data4=loadedtokenizer.texts_to_sequences(data4)
    data4=pad_sequences(data4,maxlen=120,truncating='post')
    prediction4 = loadedmodel.predict(data4)[0][0]  
    prediction4 = int(prediction4*100)
    return prediction4

In [16]:
data["positivity_score"]=data["review"].apply(positivity_score)

1/1 [==============================] - 0s 37ms/step


In [17]:
def change_rating(num):
    num = int(num*10)
    return num

In [18]:
data['rating']=data['rating'].apply(change_rating)

In [19]:
data.head()

,drugName,condition,review,rating,usefulCount,positivity_score
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",90,27,82
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",80,192,86
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",50,17,83
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",80,10,84
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",90,37,84


In [20]:
def genuinity_calculator(rating,positivity_score):
    diff = rating-positivity_score
    if diff<0:
        diff = -1*diff
    #if review and rating is almost same then 1 is label else 0 is label, for this the threshold is it should be atleast 15% similar
    if diff<=15:
        return 1
    else:
        return 0

In [21]:
# data['genuinity_label']=data['rating','positivity_score'].apply(genuinity_calculator)
data['genuinity_label'] = data.apply(lambda x: genuinity_calculator(x['rating'], x['positivity_score']), axis=1)

In [22]:
data.head(20)

,drugName,condition,review,rating,usefulCount,positivity_score,genuinity_label
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",90,27,82,1
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",80,192,86,1
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",50,17,83,0
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",80,10,84,1
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",90,37,84,1
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",20,43,16,1
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",10,5,82,0
7,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",100,32,78,0
8,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",10,11,37,0
9,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",80,1,93,1


In [24]:
print(data['review'][6])

"He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I&#039;m pregnant."


In [25]:
finaldata1 = data[data['genuinity_label']==1]

In [26]:
np.unique(finaldata1['genuinity_label'])

array([1], dtype=int64)

## Repeating the same process on another 1000 rows

In [27]:
data2 = pd.read_csv('drugsComTrain_raw.tsv',sep="\t")
data2 = data2.iloc[1000:2000,:]
data2.drop(["Unnamed: 0","date"],axis=1,inplace=True)
data2.head()

,drugName,condition,review,rating,usefulCount
1000,Diclofenac,Pain,"""I used Voltaren gel on some sore joints (and ...",9.0,68
1001,Vilazodone,Depression,"""I started Viibryd -- 20 mg/day -- over a year...",8.0,57
1002,Amoxicillin / clavulanate,Strep Throat,"""second day of being sick I was diagnosed (rap...",1.0,7
1003,Paliperidone,Schizophrenia,"""I have been on paliperidone for about a year ...",2.0,5
1004,Nexplanon,Birth Control,"""I got my Nexplanon inserted in the month of J...",9.0,2


In [28]:
data2["positivity_score"]=data2["review"].apply(positivity_score)
data2['rating']=data2['rating'].apply(change_rating)
data2['genuinity_label'] = data2.apply(lambda x: genuinity_calculator(x['rating'], x['positivity_score']), axis=1)

1/1 [==============================] - 0s 30ms/step


In [29]:
data2.head(20)

,drugName,condition,review,rating,usefulCount,positivity_score,genuinity_label
1000,Diclofenac,Pain,"""I used Voltaren gel on some sore joints (and ...",90,68,26,0
1001,Vilazodone,Depression,"""I started Viibryd -- 20 mg/day -- over a year...",80,57,15,0
1002,Amoxicillin / clavulanate,Strep Throat,"""second day of being sick I was diagnosed (rap...",10,7,46,0
1003,Paliperidone,Schizophrenia,"""I have been on paliperidone for about a year ...",20,5,27,1
1004,Nexplanon,Birth Control,"""I got my Nexplanon inserted in the month of J...",90,2,69,0
1005,Vicodin ES,Pain,"""I&#039;ve had back pain for almost 3 years no...",100,15,88,1
1006,Metronidazole,Bacterial Infection,"""Had severe diarrhea, admitted in hospital. Th...",100,3,66,0
1007,Hydromorphone,Pain,"""I had this in hospital when I broke my arm. I...",100,8,76,0
1008,Zyban,Smoking Cessation,"""Started smoking at 13, up to 2 1/2 pack a day...",100,67,53,0
1009,Lexapro,Generalized Anxiety Disorde,"""After being diagnosed with depression and gen...",90,77,91,1


In [30]:
finaldata2 = data2[data2['genuinity_label']==1]
np.unique(finaldata2['genuinity_label'])

array([1], dtype=int64)

In [32]:
finaldata = pd.concat([finaldata1,finaldata2],axis=0)

In [33]:
finaldata.head()

,drugName,condition,review,rating,usefulCount,positivity_score,genuinity_label
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",90,27,82,1
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",80,192,86,1
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",80,10,84,1
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",90,37,84,1
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",20,43,16,1


In [34]:
finaldata['genuinity_label'].value_counts()

1    582
Name: genuinity_label, dtype: int64

In [36]:
finaldata.drop(['positivity_score','genuinity_label'],axis=1,inplace=True)

In [37]:
finaldata["positivity_score"]=finaldata["review"].apply(positivity_score)
finaldata['genuinity_label']=finaldata.apply(lambda x: genuinity_calculator(x['rating'], x['positivity_score']), axis=1)

1/1 [==============================] - 0s 34ms/step


In [38]:
finaldata.head()

,drugName,condition,review,rating,usefulCount,positivity_score,genuinity_label
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",90,27,82,1
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",80,192,86,1
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",80,10,84,1
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",90,37,84,1
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",20,43,16,1


In [39]:
finaldata['genuinity_label'].value_counts()

1    582
Name: genuinity_label, dtype: int64